In [49]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

year_List = list(range(2016,2022))
year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

category = ["연령별", "가구소득별"]

# 여가 관련 자료

## 일과 여가 생활 균형

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)


## 일과 여가 생활 균형 2

In [63]:
leisure2 = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [71]:
leis_sep = leisure2.iloc[:,:2]

for yr in year_List : 
    pattern = str(yr)
    globals()["leis2_{0}".format(str(yr))] = leisure2.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
    globals()["leis2_{0}".format(str(yr))] = pd.concat([leis_sep, globals()["leis2_{0}".format(str(yr))]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].replace("-",0)
    globals()["leis2_{0}".format(str(yr))].columns = globals()["leis2_{0}".format(str(yr))].iloc[0,:].to_list()
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].iloc[1:,:]
    globals()["leis2_{0}".format(str(yr))].iloc[:,2:] = globals()["leis2_{0}".format(str(yr))].iloc[:,2:].astype(float)
    # globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].set_index(["통계분류(1)","통계분류(2)"])
    globals()["leis2_{0}".format(str(yr))].iloc[:,3:] = globals()["leis2_{0}".format(str(yr))].iloc[:,3:].apply(lambda x : x * leis2_2016.iloc[:,2] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].drop("표본수",axis=1)  # 합계 부분 삭제
    globals()["leis2_{0}".format(str(yr))].iloc[:,2:] = globals()["leis2_{0}".format(str(yr))].iloc[:,2:].astype(int)
    globals()["일여가균형_{0}_{1}".format(str(yr),"연령별")] = globals()["leis2_{0}".format(str(yr))][globals()["leis2_{0}".format(str(yr))]["통계분류(1)"]=="연령별"].iloc[:,1:]
    globals()["일여가균형_{0}_{1}".format(str(yr),"가구소득별")] = globals()["leis2_{0}".format(str(yr))][globals()["leis2_{0}".format(str(yr))]["통계분류(1)"]=="가구소득별"].iloc[:,1:]


In [72]:
일여가균형_2016_가구소득별
# .filter(like="연령별", axis=0)

,통계분류(2),일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
32,100만원 미만,7,36,105,266,248,154,41
33,100~200만원,16,87,151,379,265,158,17
34,200~300만원,39,164,306,516,376,180,9
35,300~400만원,29,221,538,889,477,268,9
36,400~500만원,31,189,468,809,490,236,6
37,500~600만원,23,109,303,503,271,141,5
38,600만원 이상,22,90,226,364,226,111,8


In [46]:
leis2_2016["통계분류(2)"].unique()

array(['소계', '남성', '여성', '15~19세', '20대', '30대', '40대', '50대', '60대',
       '70세이상', '초졸 이하', '중졸', '고졸', '대졸이상', '1인', '2인', '3인이상', '미혼',
       '기혼', '사별/이혼/기타', '가구주', '가구주 아님', '상용근로자', '임시근로자', '일용근로자',
       '고용원이 있는 자영업자', '고용원이 없는 자영업자', '무급가족종사자', '기타종사자', '해당없음(무직)',
       '임금.봉급 근로자', '100만원 미만', '100~200만원', '200~300만원', '300~400만원',
       '400~500만원', '500~600만원', '600만원 이상', '대도시', '중소도시', '읍면지역', '수도권',
       '강원/제주권', '충청/세종권', '호남권', '대경권', '동남권', '서울', '부산', '대구', '인천',
       '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북',
       '경남', '제주', '장애등록', '미등록', '해당사항없음'], dtype=object)

In [48]:
leis2_2016[leis2_2016["통계분류(1)"] == "가구소득별"]

,통계분류(1),통계분류(2),표본수,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
32,가구소득별,100만원 미만,861.0,0.9,4.2,12.2,31.0,28.9,18.0,4.8
33,가구소득별,100~200만원,1075.0,1.5,8.1,14.1,35.3,24.7,14.7,1.6
34,가구소득별,200~300만원,1594.0,2.5,10.3,19.2,32.4,23.6,11.3,0.6
35,가구소득별,300~400만원,2437.0,1.2,9.1,22.1,36.5,19.6,11.0,0.4
36,가구소득별,400~500만원,2231.0,1.4,8.5,21.0,36.3,22.0,10.6,0.3
37,가구소득별,500~600만원,1356.0,1.7,8.1,22.4,37.1,20.0,10.4,0.4
38,가구소득별,600만원 이상,1048.0,2.1,8.6,21.6,34.8,21.6,10.6,0.8
